In [1]:
from routingpy import OSRM
import pandas as pd

# read data to dataframe
df = pd.read_csv('delivery_time_data.csv', sep=';', decimal=',')
print(df)

# coords = [[77.683237, 12.913041], [77.813237, 13.043041]]
# coords = [[22.745049, 75.892471], [22.765049, 75.912471]]
# coords = [[75.892471, 22.745049], [75.912471, 22.765049]]
# client = OSRM()


# route = client.directions(locations=coords)
# print(route.distance)


            ID Delivery_person_ID  Delivery_person_Age  \
0      0x4607     INDORES13DEL02                    37   
1      0xb379     BANGRES18DEL02                    34   
2      0x5d6d     BANGRES19DEL01                    23   
3      0x7a6a    COIMBRES13DEL02                    38   
4      0x70a2     CHENRES12DEL01                    32   
...        ...                ...                  ...   
39258  0x1178   RANCHIRES16DEL01                    35   
39259  0x7c09      JAPRES04DEL01                    30   
39260  0x4f8d     CHENRES08DEL03                    30   
39261  0x5eee    COIMBRES11DEL01                    20   
39262  0x5fb2   RANCHIRES09DEL02                    23   

       Delivery_person_Ratings  Restaurant_latitude  Restaurant_longitude  \
0                          4.9            22.745049             75.892471   
1                          4.5            12.913041             77.683237   
2                          4.4            12.914264             77.67840

In [2]:
# convert lat/lon columns to float values
from geopy import distance

geodesic_distances = []

for x1,y1,x2,y2 in zip(df['Restaurant_longitude'], df['Restaurant_latitude'], df['Delivery_location_longitude'], df['Delivery_location_latitude']):
    # passing latitude first, then longitude for correct outcome. Gets geodesic measure, which is somewhat lower bound for distance between these two points
    geodesic_distances.append(distance.geodesic((y1,x1),(y2,x2)).km)

df['Straight line distances [km]'] = geodesic_distances

In [3]:
df['Straight line distances [km]']

0         3.020737
1        20.143737
2         1.549693
3         7.774497
4         6.197898
           ...    
39258    16.576800
39259     1.488112
39260     4.648024
39261     6.219668
39262    12.057294
Name: Straight line distances [km], Length: 39263, dtype: float64

In [4]:
# collecting road distances
client = OSRM()

#PLEASE RUN CAREFULLY PLEASE

# x1,y1,x2,y2 = df['Restaurant_longitude'][0], df['Restaurant_latitude'][0], df['Delivery_location_longitude'][0], df['Delivery_location_latitude'][0]

iter = 1
route_distances = []
exception_cases = []
for x1,y1,x2,y2 in zip(df['Restaurant_longitude'], df['Restaurant_latitude'], df['Delivery_location_longitude'], df['Delivery_location_latitude']):
    try:
        coords = [[x1,y1], [x2,y2]]
        route = client.directions(locations=coords)
        route_distances.append(route.distance/1000)
    except Exception as e:
        print(e, f"\tIter num: {iter}")
        exception_cases.append(iter)
    iter += 1

400 ({'message': 'Impossible route between points', 'code': 'NoRoute'}) 	Iter num: 23
400 ({'message': 'Impossible route between points', 'code': 'NoRoute'}) 	Iter num: 376
400 ({'message': 'Impossible route between points', 'code': 'NoRoute'}) 	Iter num: 479
400 ({'message': 'Impossible route between points', 'code': 'NoRoute'}) 	Iter num: 561
400 ({'message': 'Impossible route between points', 'code': 'NoRoute'}) 	Iter num: 765
400 ({'message': 'Impossible route between points', 'code': 'NoRoute'}) 	Iter num: 1018
400 ({'message': 'Impossible route between points', 'code': 'NoRoute'}) 	Iter num: 1021
400 ({'message': 'Impossible route between points', 'code': 'NoRoute'}) 	Iter num: 1154
400 ({'message': 'Impossible route between points', 'code': 'NoRoute'}) 	Iter num: 1407
400 ({'message': 'Impossible route between points', 'code': 'NoRoute'}) 	Iter num: 1458
400 ({'message': 'Impossible route between points', 'code': 'NoRoute'}) 	Iter num: 1507
400 ({'message': 'Impossible route bet

In [8]:
# write exception cases to file to not loose them
print(exception_cases)

[23, 376, 479, 561, 765, 1018, 1021, 1154, 1407, 1458, 1507, 1566, 1872, 1934, 1991, 2106, 2277, 2419, 2455, 2913, 2935, 2988, 3066, 3084, 3125, 3204, 3229, 3403, 3530, 3693, 3704, 3750, 3874, 3964, 4065, 4459, 4615, 4965, 5036, 5193, 5202, 5244, 5305, 5716, 6118, 6378, 6618, 6755, 6996, 7144, 7242, 7501, 7599, 8174, 8298, 8514, 9122, 9165, 9370, 9397, 9693, 9805, 10043, 10352, 10396, 10519, 10583, 10808, 10828, 11030, 11257, 11286, 11325, 11344, 11621, 11641, 11643, 11713, 11742, 11838, 12124, 12198, 12223, 12350, 12361, 12706, 13014, 13320, 13419, 13508, 13512, 13648, 13869, 14148, 14171, 14428, 14578, 14688, 14897, 14924, 14927, 15420, 15690, 15898, 15909, 16129, 16210, 16323, 16375, 16394, 16404, 16505, 16845, 16909, 16980, 17088, 17444, 17508, 18364, 18684, 19557, 19586, 19661, 19765, 20003, 20276, 20318, 20321, 20688, 20742, 20879, 21015, 21227, 21271, 21698, 21732, 21826, 21934, 21947, 21991, 22111, 22159, 22236, 22400, 22455, 22833, 22910, 22968, 23002, 23063, 23176, 23244, 233

In [9]:
# write good routes to file to not loose them
with open('good_routes_distances.txt', 'w') as f:
    for line in route_distances:
        f.write(f"{line}\n")

In [14]:
exception_cases_match_df = [x-1 for x in exception_cases]
exception_cases_match_df

[22,
 375,
 478,
 560,
 764,
 1017,
 1020,
 1153,
 1406,
 1457,
 1506,
 1565,
 1871,
 1933,
 1990,
 2105,
 2276,
 2418,
 2454,
 2912,
 2934,
 2987,
 3065,
 3083,
 3124,
 3203,
 3228,
 3402,
 3529,
 3692,
 3703,
 3749,
 3873,
 3963,
 4064,
 4458,
 4614,
 4964,
 5035,
 5192,
 5201,
 5243,
 5304,
 5715,
 6117,
 6377,
 6617,
 6754,
 6995,
 7143,
 7241,
 7500,
 7598,
 8173,
 8297,
 8513,
 9121,
 9164,
 9369,
 9396,
 9692,
 9804,
 10042,
 10351,
 10395,
 10518,
 10582,
 10807,
 10827,
 11029,
 11256,
 11285,
 11324,
 11343,
 11620,
 11640,
 11642,
 11712,
 11741,
 11837,
 12123,
 12197,
 12222,
 12349,
 12360,
 12705,
 13013,
 13319,
 13418,
 13507,
 13511,
 13647,
 13868,
 14147,
 14170,
 14427,
 14577,
 14687,
 14896,
 14923,
 14926,
 15419,
 15689,
 15897,
 15908,
 16128,
 16209,
 16322,
 16374,
 16393,
 16403,
 16504,
 16844,
 16908,
 16979,
 17087,
 17443,
 17507,
 18363,
 18683,
 19556,
 19585,
 19660,
 19764,
 20002,
 20275,
 20317,
 20320,
 20687,
 20741,
 20878,
 21014,
 21226,
 212

In [21]:
# drop unroutable roads
filtered_df = df.drop(exception_cases_match_df).reset_index()

In [22]:
# add column with road distances
filtered_df["Road distances [km]"] = route_distances

In [23]:
# write to csv
filtered_df.to_csv('road_delivery_data.csv')